# Non-dominated Sorting Genetic Algorithm II (NSGA-II)

It gives several builds at the end of it

In [1]:
# required for running NSGA-ii

# !pip install pymoo

In [2]:
import numpy as np
import pandas as pd
from pymoo.core.problem import ElementwiseProblem
from pymoo.optimize import minimize
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.termination import get_termination

In [3]:
path = "../data/parts/"

# Load datasets
games_df = pd.read_csv("../data/games/top300.csv")

cpu_df = pd.read_csv(path + "CPU_Data.csv")
gpu_df = pd.read_csv(path + "GPU_Data.csv")
ram_df = pd.read_csv(path + "RAM_Data.csv")

mobo_df = pd.read_csv(path + "MOBO_Data.csv")
psu_df = pd.read_csv(path + "PSU_Data.csv")
case_df = pd.read_csv(path + "Case_Data.csv")

# storage_df = pd.read_csv(data_path + "Storage_Data.csv")

def safe_to_numeric(value):
    try:
        return pd.to_numeric(value)
    except (ValueError, TypeError):
        return value  

# Convert columns to numeric where possible
games_df = games_df.apply(safe_to_numeric)
cpu_df = cpu_df.apply(safe_to_numeric)
gpu_df = gpu_df.apply(safe_to_numeric)
ram_df = ram_df.apply(safe_to_numeric)
case_df = case_df.apply(safe_to_numeric)
mobo_df = mobo_df.apply(safe_to_numeric)
psu_df = psu_df.apply(safe_to_numeric)

# storage_df = storage_df.apply(safe_to_numeric)

In [4]:
# Precompute valid CPU-MOBO pairs
valid_cpu_mobo_pairs = cpu_df.merge(
    mobo_df, on="Socket", suffixes=("_cpu", "_mobo")
).reset_index(drop=True)

# Set ram options
ram_stick_options = [1, 2, 4]

In [20]:
# Maximize budget
class PCBuildProblem(ElementwiseProblem):
    def __init__(self, budget, required_cpu_score, required_gpu_score, required_ram_gb):
        self.budget = budget
        self.required_cpu_score = required_cpu_score
        self.required_gpu_score = required_gpu_score
        self.required_ram_gb = max(required_ram_gb, 8)  # Enforce minimum 8 GB RAM

        super().__init__(
            n_var=6,  # [cpu_mobo, gpu, ram, ram_stick_count, psu, case]
            n_obj=3,  # Maximize CPU score, GPU score, dual channel
            n_constr=8,  # Now includes CPU and GPU score requirements
            xl=np.array([0, 0, 0, 0, 0, 0]),
            xu=np.array([
                len(valid_cpu_mobo_pairs) - 1,
                len(gpu_df) - 1,
                len(ram_df) - 1,
                len(ram_stick_options) - 1,
                len(psu_df) - 1,
                len(case_df) - 1
            ])
        )

    def _evaluate(self, x, out, *args, **kwargs):
        cpu_mobo = valid_cpu_mobo_pairs.iloc[int(x[0])]
        gpu = gpu_df.iloc[int(x[1])]
        ram = ram_df.iloc[int(x[2])]
        sticks = ram_stick_options[int(x[3])]
        psu = psu_df.iloc[int(x[4])]
        case = case_df.iloc[int(x[5])]

        total_ram = ram['Capacity (GB)'] * sticks
        total_price = (
            cpu_mobo['Price_cpu'] + cpu_mobo['Price_mobo'] +
            gpu['Price'] + psu['Price'] + case['Price'] +
            ram['Price'] * sticks
        )

        cpu_score = cpu_mobo['Score']
        gpu_score = gpu['Score']

        # Constraint checks
        wattage_ok = psu['Wattage'] >= gpu['Recommended Power']
        case_ok = str(cpu_mobo['Size']) in str(case['Size'])
        ram_slot_ok = sticks <= cpu_mobo['RAM Slot']
        ddr_ok = str(ram['DDR']) == str(cpu_mobo['DDR'])
        min_ram_ok = total_ram >= self.required_ram_gb
        budget_ok = total_price <= self.budget
        cpu_ok = cpu_score >= self.required_cpu_score
        gpu_ok = gpu_score >= self.required_gpu_score

        # Dual-channel preference
        dual_channel_score = 1 if sticks in [2, 4] else 0

        # Objectives (maximize → negate)
        out["F"] = [
            -cpu_score,
            -gpu_score,
            -dual_channel_score
        ]

        # Constraints: must be <= 0 to be feasible
        out["G"] = [
            0 if wattage_ok else 1,
            0 if case_ok else 1,
            0 if ram_slot_ok else 1,
            0 if ddr_ok else 1,
            0 if min_ram_ok else 1,
            0 if budget_ok else 1,
            0 if cpu_ok else 1,
            0 if gpu_ok else 1 
        ]

In [23]:
budget = 2500
required_cpu_score = 10000
required_gpu_score = 13000
required_ram_gb = 16

problem = PCBuildProblem(budget, required_cpu_score, required_gpu_score, required_ram_gb)

# Run NSGA-II
algorithm = NSGA2(pop_size=100)
termination = get_termination("n_gen", 50)
res = minimize(problem, algorithm, termination, seed=1, save_history=True, verbose=True)

# Display results
for i, sol in enumerate(res.X):
    cpu_mobo = valid_cpu_mobo_pairs.iloc[int(sol[0])]
    gpu = gpu_df.iloc[int(sol[1])]
    ram = ram_df.iloc[int(sol[2])]
    sticks = ram_stick_options[int(sol[3])]
    psu = psu_df.iloc[int(sol[4])]
    case = case_df.iloc[int(sol[5])]

    total_price = (
        cpu_mobo['Price_cpu'] + cpu_mobo['Price_mobo'] +
        gpu['Price'] + psu['Price'] + case['Price'] +
        ram['Price'] * sticks
    )

    print(f"\nBuild {i + 1}:")
    print("CPU:", cpu_mobo['Name_cpu'])
    print("MOBO:", cpu_mobo['Name_mobo'])
    print("GPU:", gpu['Name'])
    print("RAM:", f"{sticks}x {ram['Name']}")
    print("PSU:", psu['Name'])
    print("Case:", case['Name'])
    print(f"Total Price: RM{total_price:.2f}")
    print("CPU Score:", -res.F[i][0])
    print("GPU Score:", -res.F[i][1])

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |      100 |      1 |  0.000000E+00 |  2.8400000000 |             - |             -
     2 |      200 |      2 |  0.000000E+00 |  1.8900000000 |  1.0000000000 |         ideal
     3 |      300 |      2 |  0.000000E+00 |  1.4300000000 |  0.000000E+00 |             f
     4 |      400 |      2 |  0.000000E+00 |  0.9300000000 |  0.6161473088 |         ideal
     5 |      500 |      1 |  0.000000E+00 |  0.8500000000 |  9.600000E+02 |         ideal
     6 |      600 |      2 |  0.000000E+00 |  0.7600000000 |  1.0000000000 |         ideal
     7 |      700 |      3 |  0.000000E+00 |  0.6200000000 |  0.000000E+00 |             f
     8 |      800 |      5 |  0.000000E+00 |  0.3400000000 |  0.000000E+00 |             f
     9 |      900 |     12 |  0.000000E+00 |  0.000000E+00 |  0.0898312118 |             f
    10 |     1000 |     15 |  0.000000E+00 |  0.000000E+00 |  0.3912175649 |         nadir

In [ ]:
class PCBuildCostMinProblem(ElementwiseProblem):
    def __init__(self, budget, required_cpu_score, required_gpu_score, required_ram_gb):
        self.budget = budget
        self.required_cpu_score = required_cpu_score
        self.required_gpu_score = required_gpu_score
        self.required_ram_gb = max(required_ram_gb, 8)  # Enforce minimum 8GB RAM

        super().__init__(
            n_var=6,  # [cpu_mobo_index, gpu_index, ram_index, ram_sticks, psu_index, case_index]
            n_obj=1,  # minimize total price
            n_constr=6,  # CPU, GPU, PSU, RAM, case, and budget
            xl=np.array([0, 0, 0, 0, 0, 0]),
            xu=np.array([
                len(valid_cpu_mobo_pairs) - 1,
                len(gpu_df) - 1,
                len(ram_df) - 1,
                len(ram_stick_options) - 1,
                len(psu_df) - 1,
                len(case_df) - 1
            ])
        )

    def _evaluate(self, x, out, *args, **kwargs):
        cpu_mobo = valid_cpu_mobo_pairs.iloc[int(x[0])]
        gpu = gpu_df.iloc[int(x[1])]
        ram = ram_df.iloc[int(x[2])]
        sticks = ram_stick_options[int(x[3])]
        psu = psu_df.iloc[int(x[4])]
        case = case_df.iloc[int(x[5])]

        total_ram = ram['Capacity (GB)'] * sticks
        total_price = (
            cpu_mobo['Price_cpu'] + cpu_mobo['Price_mobo'] +
            gpu['Price'] + psu['Price'] + case['Price'] +
            ram['Price'] * sticks
        )

        cpu_score = cpu_mobo['Score']
        gpu_score = gpu['Score']

        # Constraints
        wattage_ok = psu['Wattage'] >= gpu['Recommended Power']
        case_ok = str(cpu_mobo['Size']) in str(case['Size'])
        ram_slot_ok = sticks <= cpu_mobo['RAM Slot']
        ddr_ok = str(ram['DDR']) == str(cpu_mobo['DDR'])
        min_ram_ok = total_ram >= self.required_ram_gb
        cpu_ok = cpu_score >= self.required_cpu_score
        gpu_ok = gpu_score >= self.required_gpu_score
        budget_ok = total_price <= self.budget

        out["F"] = [total_price]  # pure cost minimization
        out["G"] = [
            0 if cpu_ok else 1,
            0 if gpu_ok else 1,
            0 if wattage_ok else 1,
            0 if case_ok else 1,
            0 if ram_slot_ok and ddr_ok and min_ram_ok else 1,
            0 if budget_ok else 1  # <-- HARD budget constraint
        ]


In [11]:
budget = 2500
required_cpu_score = 6000
required_gpu_score = 8000
required_ram_gb = 16

problem = PCBuildCostMinProblem(budget, required_cpu_score, required_gpu_score, required_ram_gb)

# Run NSGA-II
algorithm = NSGA2(pop_size=100)
termination = get_termination("n_gen", 50)
res = minimize(problem, algorithm, termination, seed=1, save_history=True, verbose=True)

# Display results
for i, sol in enumerate(res.X):
    cpu_mobo = valid_cpu_mobo_pairs.iloc[int(sol[0])]
    gpu = gpu_df.iloc[int(sol[1])]
    
    cpu_score = cpu_mobo['Score']
    gpu_score = gpu['Score']
    
    print(f"\nBuild {i + 1}:")
    print("CPU:", valid_cpu_mobo_pairs.iloc[int(sol[0])]['Name_cpu'])
    print("MOBO:", valid_cpu_mobo_pairs.iloc[int(sol[0])]['Name_mobo'])
    print("GPU:", gpu_df.iloc[int(sol[1])]['Name'])
    print("RAM:", f"{ram_stick_options[int(sol[3])]}x {ram_df.iloc[int(sol[2])]['Name']}")
    print("PSU:", psu_df.iloc[int(sol[4])]['Name'])
    print("Case:", case_df.iloc[int(sol[5])]['Name'])
    print("Total Price:", res.F[i][0])
    print("CPU Score:", cpu_score)
    print("GPU Score:", gpu_score)

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |      100 |      1 |  0.000000E+00 |  1.4900000000 |             - |             -
     2 |      200 |      1 |  0.000000E+00 |  0.7300000000 |  3.600000E+02 |         ideal
     3 |      300 |      1 |  0.000000E+00 |  0.4200000000 |  0.000000E+00 |             f
     4 |      400 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     5 |      500 |      1 |  0.000000E+00 |  0.000000E+00 |  1.840000E+02 |         ideal
     6 |      600 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     7 |      700 |      1 |  0.000000E+00 |  0.000000E+00 |  3.000000E+01 |         ideal
     8 |      800 |      1 |  0.000000E+00 |  0.000000E+00 |  3.200000E+02 |         ideal
     9 |      900 |      1 |  0.000000E+00 |  0.000000E+00 |  3.000000E+01 |         ideal
    10 |     1000 |      1 |  0.000000E+00 |  0.000000E+00 |  1.000000E+01 |         ideal

In [ ]:
def buildPC_NSGA(game_names: list[str], budget: float, mode: str = "performance"):
    # Normalize game names for comparison
    game_names_lower = [name.lower() for name in game_names]
    matched_games = games_df[games_df["name"].str.lower().isin(game_names_lower)]

    if matched_games.empty:
        raise ValueError("None of the specified games were found in the dataset.")

    missing = [name for name in game_names if name.lower() not in matched_games["name"].str.lower().values]
    if missing:
        print(f"Warning: These games were not found and will be skipped: {', '.join(missing)}")

    try:
        required_cpu_score = matched_games["CPU"].astype(float).max()
        required_gpu_score = matched_games["GPU"].astype(float).max()
        required_ram_gb = matched_games["memory"].astype(int).max()
    except Exception as e:
        raise ValueError(f"Failed to extract requirements from one or more games: {e}")

    print(f"Aggregated Requirements — CPU: {required_cpu_score}, GPU: {required_gpu_score}, RAM: {required_ram_gb}GB")

    if mode == "performance":
        return PCBuildProblem(
            budget=budget,
            required_cpu_score=required_cpu_score,
            required_gpu_score=required_gpu_score,
            required_ram_gb=required_ram_gb
        )
    elif mode == "cost":
        return PCBuildCostMinProblem(
            budget=budget,
            required_cpu_score=required_cpu_score,
            required_gpu_score=required_gpu_score,
            required_ram_gb=required_ram_gb
        )
    else:
        raise ValueError(f"Unknown mode '{mode}'. Choose 'performance' or 'cost'.")


In [ ]:
# Performance-focused build (maximize CPU/GPU/dual-channel)
problem = buildPC_NSGA(["Elden Ring", "Cyberpunk 2077"], budget=3000, mode="performance")

# Cost-minimized build (satisfy constraints with lowest price)
# problem = buildPC_NSGA(["Elden Ring", "Cyberpunk 2077"], budget=3000, mode="cost")

algorithm = NSGA2(pop_size=100)
termination = get_termination("n_gen", 50)
res = minimize(problem, algorithm, termination, seed=1, save_history=True, verbose=True)

# Display results
for i, sol in enumerate(res.X):
    cpu_mobo = valid_cpu_mobo_pairs.iloc[int(sol[0])]
    gpu = gpu_df.iloc[int(sol[1])]
    
    cpu_score = cpu_mobo['Score']
    gpu_score = gpu['Score']
    
    print(f"\nBuild {i + 1}:")
    print("CPU:", valid_cpu_mobo_pairs.iloc[int(sol[0])]['Name_cpu'])
    print("MOBO:", valid_cpu_mobo_pairs.iloc[int(sol[0])]['Name_mobo'])
    print("GPU:", gpu_df.iloc[int(sol[1])]['Name'])
    print("RAM:", f"{ram_stick_options[int(sol[3])]}x {ram_df.iloc[int(sol[2])]['Name']}")
    print("PSU:", psu_df.iloc[int(sol[4])]['Name'])
    print("Case:", case_df.iloc[int(sol[5])]['Name'])
    print("Total Price:", res.F[i][0])
    print("CPU Score:", cpu_score)
    print("GPU Score:", gpu_score)